<a href="https://colab.research.google.com/github/akshaypt7/amazon_sales_prediction/blob/main/my_project_01_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is part 2 of the project. Here we make improvements that we have learnt from part 1 of the project.

One of the mistakes we have done in the first part was that there was data leakage. 

**When the sales are made for each asins in a day, we are not considering the asins that were not sold that day. so the thing is we need to add ASINS even when there was no sale made for that. We can give the quantity as 0. **


How do we do this?

When we read data each month, we see what are the unique ASINs, then these asins should be added to each days, even for the days when these asins are not sold. 
but what other details can we give, what about promotions and other details, we can take the entire row not just the ASIN, but what if there are two rows with coupon and without coupon, if we can see which row is closer to that, that is with coupon or without coupon we can copy that coupon.

one another thing to note is we can even look at working this for the enitre dataframe at once, rather than going month by month. The reason is what if for one month the product was laucnhed but on second month there was no sales, even though that product was available. this means that for all those we wont even mention the asin, but on the third month we mention again.

- we can change the continous and categorical variables that we have defined.

One another mistake that we had done in part 1 was that we had only taken 679 data points as the validation set, we have used 10,500 as the train set. we should have atleast taken 3000 units as validation set.

This was the reason we had high training error and less validation error. This makes sense why we also had high OOB_error and less validation error, as oob_error is calculated with atleast 30% data.

Next thing we learnt is we need to understand what metric we have to use.

**We can use both classification and regression. When we use both then we need to understand what has to be used as the metric.**

Also if we are considering accuracy or even other metrics we have to create a baseline model which predicts the majority class for all the data points.

Learn more about metrics.

One another thing what we can note is, we can convert 1.4 to 2, if we are going with regression. Here we need to come up with something like np.where(). We can see what is the performance at different thresholds. 

We should also try to create the functions without copying and trying on a smaller set to know if it works.

In [ ]:
#hide
!pip install -Uqq fastbook kaggle waterfallcharts treeinterpreter dtreeviz
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 33.9MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 1.2MB 45.3MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 194kB 50.5MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
Mounted at /content/gdrive


In [ ]:
#hide
from fastbook import *
# from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [ ]:
import pandas as pd

In [ ]:
import os
files = os.listdir('/content/gdrive/MyDrive/my_project_01')


In [ ]:
dict_files = {}
for i,file in enumerate(files):
  try:
    # dict_files[i] = pd.read_csv( '/content/drive/MyDrive/my_project_01/' +str(file),low_memory=False)
    dict_files[i] = pd.read_csv( '/content/gdrive/MyDrive/my_project_01/' +str(file),low_memory=False)
  except:
    # dict_files[i] = pd.read_excel( '/content/drive/MyDrive/my_project_01/' +str(file))
    dict_files[i] = pd.read_excel( '/content/gdrive/MyDrive/my_project_01/' +str(file))


We have to take in Invoice Amount instead of Principal amount (since it helps to sepearate canceled and refund orders)

In [ ]:
df_dicts = {}

for i in range(len(dict_files)):
  df_dicts[i] = dict_files[i][['Order Date','Quantity','Asin','Invoice Amount','Item Promo Discount']]


Testing out with 1 dataframe

In [ ]:
df_copy = df.copy()

In [ ]:
df = df_dicts[1]
df

,Order Date,Quantity,Asin,Invoice Amount,Item Promo Discount
0,2021-02-01 17:28:51,1,B07TM3LRVB,1135.00,0.00
1,2021-02-04 09:18:42,1,B07TM3LRVB,1078.25,-56.75
2,2021-02-12 00:30:53,1,B07TM3LRVB,1078.25,-56.75
3,2021-02-13 10:30:28,1,B0849NLNTQ,1599.00,0.00
4,2021-02-13 12:35:00,1,B0849NLNTQ,1519.05,-79.95
5,2021-02-14 18:47:44,1,B07KYFHTGF,1895.25,-99.75
6,2021-02-14 21:49:20,1,B07TM3LRVB,1078.25,-56.75
7,2021-02-18 07:55:57,1,B0849NLNTQ,1599.00,0.00
8,2021-02-24 14:11:53,1,B07TM3LRVB,1078.25,-56.75
9,2021-01-24 16:47:48,1,B0849NLNTQ,-1599.00,0.00


In [ ]:
df_copy['Date'] = df['Order Date'].dt.date

In [ ]:
df_copy.drop(['Order Date'], inplace = True, axis = 1)

In [ ]:
df_copy

,Quantity,Asin,Invoice Amount,Item Promo Discount,Date
0,1,B07TM3LRVB,1135.00,0.00,2021-02-01
1,1,B07TM3LRVB,1078.25,-56.75,2021-02-04
2,1,B07TM3LRVB,1078.25,-56.75,2021-02-12
3,1,B0849NLNTQ,1599.00,0.00,2021-02-13
4,1,B0849NLNTQ,1519.05,-79.95,2021-02-13
5,1,B07KYFHTGF,1895.25,-99.75,2021-02-14
6,1,B07TM3LRVB,1078.25,-56.75,2021-02-14
7,1,B0849NLNTQ,1599.00,0.00,2021-02-18
8,1,B07TM3LRVB,1078.25,-56.75,2021-02-24
9,1,B0849NLNTQ,-1599.00,0.00,2021-01-24


In [ ]:
df_copy.groupby(['Date', 'Asin','Invoice Amount','Item Promo Discount'])['Quantity'].sum()

Date        Asin        Invoice Amount  Item Promo Discount
2021-01-24  B0849NLNTQ  -1599.00         0.00                  1
2021-01-28  B08332221J  -1499.00         0.00                  1
2021-02-01  B07TM3LRVB   1135.00         0.00                  1
2021-02-04  B07TM3LRVB   1078.25        -56.75                 1
2021-02-12  B07TM3LRVB   1078.25        -56.75                 1
2021-02-13  B0849NLNTQ   1519.05        -79.95                 1
                         1599.00         0.00                  1
2021-02-14  B07KYFHTGF   1895.25        -99.75                 1
            B07TM3LRVB   1078.25        -56.75                 1
2021-02-18  B0849NLNTQ   1599.00         0.00                  1
2021-02-24  B07TM3LRVB   1078.25        -56.75                 1
Name: Quantity, dtype: int64

In [ ]:
cond  = df_copy['Invoice Amount'] > 0

In [ ]:
df = df_copy.where(cond )

In [ ]:
df

,Quantity,Asin,Invoice Amount,Item Promo Discount,Date
0,1.0,B07TM3LRVB,1135.00,0.00,2021-02-01
1,1.0,B07TM3LRVB,1078.25,-56.75,2021-02-04
2,1.0,B07TM3LRVB,1078.25,-56.75,2021-02-12
3,1.0,B0849NLNTQ,1599.00,0.00,2021-02-13
4,1.0,B0849NLNTQ,1519.05,-79.95,2021-02-13
5,1.0,B07KYFHTGF,1895.25,-99.75,2021-02-14
6,1.0,B07TM3LRVB,1078.25,-56.75,2021-02-14
7,1.0,B0849NLNTQ,1599.00,0.00,2021-02-18
8,1.0,B07TM3LRVB,1078.25,-56.75,2021-02-24
9,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.dropna(how='all',inplace=True)

In [ ]:
df_1 = pd.DataFrame(df.groupby(['Date', 'Asin','Invoice Amount','Item Promo Discount'])['Quantity'].sum())

In [ ]:
df_1['Date'] = pd.to_datetime(df_1['Date'])

In [ ]:
df_1 = df_1.reset_index()

In [ ]:
df_1

,Date,Asin,Invoice Amount,Item Promo Discount,Quantity
0,2021-02-01,B07TM3LRVB,1135.00,0.00,1.0
1,2021-02-04,B07TM3LRVB,1078.25,-56.75,1.0
2,2021-02-12,B07TM3LRVB,1078.25,-56.75,1.0
3,2021-02-13,B0849NLNTQ,1519.05,-79.95,1.0
4,2021-02-13,B0849NLNTQ,1599.00,0.00,1.0
5,2021-02-14,B07KYFHTGF,1895.25,-99.75,1.0
6,2021-02-14,B07TM3LRVB,1078.25,-56.75,1.0
7,2021-02-18,B0849NLNTQ,1599.00,0.00,1.0
8,2021-02-24,B07TM3LRVB,1078.25,-56.75,1.0


In [ ]:
df_2 = df_1.set_index('Date')

In [ ]:
df_2.loc['2021-02-12']

Asin                   B07TM3LRVB
Invoice Amount            1078.25
Item Promo Discount        -56.75
Quantity                        1
Name: 2021-02-12 00:00:00, dtype: object

In [ ]:
df_2.loc['2021-02-12']['Asin']

'B07TM3LRVB'